In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
import ndjson

from quillnlp.distillation.augmentation import augment_data

PREFIX = "eatingmeat_but_xl"

if "meat" in PREFIX:
    PROMPT = "Large amounts of meat consumption are harming the environment, "
elif "junkfood" in PREFIX:
    PROMPT = "Schools should not allow junk food to be sold on campus "

train_file = f"../data/interim/{PREFIX}_train_withprompt.ndjson"
dev_file = f"../data/interim/{PREFIX}_dev_withprompt.ndjson"
test_file = f"../data/interim/{PREFIX}_test_withprompt.ndjson"

with open(train_file) as i:
    train_data = ndjson.load(i)
    
with open(dev_file) as i:
    dev_data = ndjson.load(i)
    
with open(test_file) as i:
    test_data = ndjson.load(i)
    
augmented_data = augment_data([x["text"] for x in train_data], prompt=PROMPT, n_iter=10)

Iteration 0: masking
Iteration 0: n-gram sampling
Iteration 1: masking
Iteration 1: n-gram sampling
Iteration 2: masking
Iteration 2: n-gram sampling
Iteration 3: masking
Iteration 3: n-gram sampling
Iteration 4: masking
Iteration 4: n-gram sampling
Iteration 5: masking
Iteration 5: n-gram sampling
Iteration 6: masking
Iteration 6: n-gram sampling
Iteration 7: masking
Iteration 7: n-gram sampling
Iteration 8: masking
Iteration 8: n-gram sampling
Iteration 9: masking
Iteration 9: n-gram sampling


In [3]:
len(augmented_data)

16660

In [4]:
list(augmented_data)[-100:]

['of the American',
 'Large amounts of meat consumption are harming the environment, but harming [MASK] [MASK] jobs!',
 'Large amounts of meat consumption are harming the environment, but it could hurt American businesses and jobs.',
 'Large amounts of meat consumption are harming the environment, but [MASK] meat production will [MASK] us economically because it is one of the major American domestic industries.',
 'Large amounts of meat consumption are harming the environment, [MASK] it is significant to consume meat in moderation.',
 'Large amounts of meat consumption are harming the environment, but the meat industry is rising  reducing [MASK]could damage [MASK] important position that the infrastructure taking industry  economy but loss of precious jobs.',
 'Large amounts of meat consumption are harming the environment, but reducing meat consumption [MASK] harm [MASK] economy and take away [MASK].',
 'Large amounts of meat consumption are harming the environment, but helps the econo

In [5]:
import json
import torch

from quillnlp.models.bert.models import get_bert_classifier

BERT_MODEL = 'bert-large-uncased'
MODEL_FILE = "/tmp/model.bin"
LABEL_FILE = "/tmp/labels.json"

device = "cuda" if torch.cuda.is_available() else "cpu"

with open(LABEL_FILE) as i:
    label2idx = json.load(i)

idx2label = {v:k for k,v in label2idx.items()}
print(idx2label)

model = get_bert_classifier(BERT_MODEL, len(label2idx), model_file=MODEL_FILE, device=device)
model.eval()

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

{0: 'Change without mentioning consumption', 1: 'Less meat consumption could harm economy and cut jobs', 2: 'The meat industry is important/thriving and/or exports/demand increasing', 3: 'Eating meat is necessary for good nutrition', 4: 'Eating meat is part of culture/tradition', 5: 'Meat creates jobs and benefits economy', 6: "Outside of article's scope", 7: 'People will or should still eat meat', 8: 'Flexitarian w/o connection to environment or jobs', 9: 'Flexitarians benefit environment', 10: 'Meat consumption harms environment'}


I1101 21:32:47.638557 139627641636672 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /home/yves/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
I1101 21:32:47.640964 139627641636672 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 11,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1101 21:32:48.147

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm(torch.Size([1024]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm(torch.Size([1024]), eps=1e-12, el

In [6]:
from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary, get_data_loader

MAX_SEQ_LENGTH = 100
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
    
label2idx = create_label_vocabulary(train_data)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]

train_dataloader = get_data_loader(preprocess(train_data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH), BATCH_SIZE)
dev_dataloader = get_data_loader(preprocess(dev_data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH), BATCH_SIZE)
test_dataloader = get_data_loader(preprocess(test_data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH), BATCH_SIZE, shuffle=False)

synthetic_data = [{"text": text, "label": idx2label[0]} for text in augmented_data]
synthetic_dataloader = get_data_loader(preprocess(synthetic_data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH), BATCH_SIZE, shuffle=False)


I1101 21:32:56.954175 139627641636672 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I1101 21:32:57.945175 139627641636672 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I1101 21:32:58.531907 139627641636672 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f

In [7]:
from quillnlp.models.bert.train import evaluate

_, probabilities, _, _ = evaluate(model, synthetic_dataloader, device)

../quillnlp/models/bert/train.py:145: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = softmax(logits.to('cpu'))


In [8]:
from quillnlp.models.bert.train import evaluate
from sklearn.metrics import precision_recall_fscore_support, classification_report

device="cpu"
model = get_bert_classifier(BERT_MODEL, len(label2idx), model_file=MODEL_FILE, device=device)
model.eval()
_, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)

print("Test performance:", precision_recall_fscore_support(test_correct, test_predicted, average="micro"))
print(classification_report(test_correct, test_predicted, target_names=target_names))

I1101 21:37:53.081842 139627641636672 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /home/yves/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
I1101 21:37:53.085293 139627641636672 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 11,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1101 21:37:53.598


Test performance: (0.9329268292682927, 0.9329268292682927, 0.9329268292682927, None)
                                                                          precision    recall  f1-score   support

                                   Change without mentioning consumption       0.00      0.00      0.00         1
                   Less meat consumption could harm economy and cut jobs       1.00      0.98      0.99        42
The meat industry is important/thriving and/or exports/demand increasing       0.95      1.00      0.97        18
                             Eating meat is necessary for good nutrition       0.67      0.67      0.67         6
                                Eating meat is part of culture/tradition       0.86      1.00      0.93        19
                                  Meat creates jobs and benefits economy       0.97      0.97      0.97        40
                                              Outside of article's scope       0.92      1.00      0.96        11
 

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
import numpy as np

from quillnlp.models.spacy.train import train_language

train_data_probs = []
for item in train_data:
    probs = [0]*len(label2idx)
    probs[label2idx[item["label"]]] = 1.0
    train_data_probs.append({"text": item["text"], "probabilities": probs})

synthetic_data = [{"text": item["text"], "probabilities": probs} 
                  for (item, probs) in zip(synthetic_data, probabilities)]

for x in synthetic_data[:10]:
    print(x)
    print(idx2label[np.argmax(x["probabilities"]).item()])


{'text': 'and adopt', 'probabilities': tensor([0.0345, 0.0496, 0.0592, 0.0111, 0.0837, 0.0900, 0.4133, 0.0612, 0.0937,
        0.0461, 0.0575])}
Outside of article's scope
{'text': 'Large amounts of meat consumption are harming the environment, but it would employment for many people.', 'probabilities': tensor([1.1551e-03, 1.6722e-02, 3.6214e-03, 8.1387e-04, 1.3814e-03, 9.7118e-01,
        1.3535e-03, 8.0166e-04, 6.4680e-04, 8.1583e-04, 1.5040e-03])}
Meat creates jobs and benefits economy
{'text': 'Large amounts of meat consumption are harming the environment, but it can hurt the job [MASK].', 'probabilities': tensor([6.5095e-04, 9.8797e-01, 7.0250e-04, 5.0583e-04, 7.0923e-04, 5.2384e-03,
        1.4767e-03, 5.8306e-04, 4.1155e-04, 8.7624e-04, 8.7497e-04])}
Less meat consumption could harm economy and cut jobs
{'text': 'also contributes a significant amount', 'probabilities': tensor([0.0174, 0.0196, 0.4011, 0.0097, 0.0225, 0.4280, 0.0414, 0.0148, 0.0183,
        0.0081, 0.0192])}
Meat 

In [10]:
train_language(train_data_probs, dev_data, test_data, idx2label, "en", "/tmp/spacy_junkfood_but_xl_baseline/")


I1101 21:38:20.369052 139627641636672 train.py:102] Start training for en
348it [00:13, 23.70it/s]
I1101 21:38:34.181461 139627641636672 train.py:145] 18.798	0.380
I1101 21:38:34.203792 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl_baseline
255it [00:13, 13.81it/s]
I1101 21:38:47.984197 139627641636672 train.py:145] 5.289	0.454
I1101 21:38:48.005553 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl_baseline
201it [00:14, 13.67it/s]
I1101 21:39:02.869814 139627641636672 train.py:145] 1.832	0.667
I1101 21:39:02.891378 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl_baseline
164it [00:11, 14.87it/s]
I1101 21:39:15.010812 139627641636672 train.py:145] 1.803	0.694
I1101 21:39:15.031955 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl_baseline
141it [00:09, 14.70it/s]
I1101 21:39:24.784155 139627641636672 train.py:145] 0.407	0.722
I1101 21:39:24.805138 139627641636672 train.py:154] Saved model to /

{'acc': 0.8658536585365854}

In [11]:
train_language(train_data_probs + synthetic_data, dev_data, test_data, idx2label, "en", "/tmp/spacy_junkfood_but_xl/")

I1101 21:40:31.811460 139627641636672 train.py:102] Start training for en
1752it [01:53, 15.44it/s]
I1101 21:42:26.058845 139627641636672 train.py:145] 21.898	0.731
I1101 21:42:26.080798 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl
622it [01:01, 10.11it/s]
I1101 21:43:27.766276 139627641636672 train.py:145] 0.115	0.769
I1101 21:43:27.788508 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl
568it [00:57,  9.87it/s]
I1101 21:44:25.084761 139627641636672 train.py:145] 0.067	0.769
I1101 21:44:25.106529 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl
568it [00:57,  9.78it/s]
I1101 21:45:22.743580 139627641636672 train.py:145] 0.057	0.806
I1101 21:45:22.765555 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl
568it [00:57,  9.80it/s]
I1101 21:46:20.576286 139627641636672 train.py:145] 0.054	0.815
I1101 21:46:20.597841 139627641636672 train.py:154] Saved model to /tmp/spacy_junkfood_but_xl
568it [00

{'acc': 0.9085365853658537}